# Basket Analysis

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('transaction.csv')
df.head()

,ReceiptId,Lv2Catagory,Lv3Catagory,Qty,StoreName,Date,AccountNum
0,2020101-1105832,40000 - LIFESTYLES,42000 - IT & GADGET ACCESSORIES,1.0,PM 4PM,2017-12-21,9.200012e+09
1,2020101-1021987,40000 - LIFESTYLES,42000 - IT & GADGET ACCESSORIES,1.0,PM 4PM,2016-08-13,9.200020e+09
2,2020101-1080992,40000 - LIFESTYLES,42000 - IT & GADGET ACCESSORIES,1.0,PM 4PM,2017-07-13,1.101501e+09
3,2020103-1040876,40000 - LIFESTYLES,42000 - IT & GADGET ACCESSORIES,1.0,PM 4PM,2017-01-13,2.795000e+03
4,2020101-1071109,40000 - LIFESTYLES,42000 - IT & GADGET ACCESSORIES,1.0,PM 4PM,2017-05-19,1.379940e+05


In [3]:
display(df.info())
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358747 entries, 0 to 358746
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ReceiptId    358747 non-null  object 
 1   Lv2Catagory  358747 non-null  object 
 2   Lv3Catagory  358079 non-null  object 
 3   Qty          358747 non-null  float64
 4   StoreName    358747 non-null  object 
 5   Date         358747 non-null  object 
 6   AccountNum   358747 non-null  float64
dtypes: float64(2), object(5)
memory usage: 19.2+ MB


None

,Qty,AccountNum
count,358747.000000,3.587470e+05
mean,1.279977,6.602408e+09
std,4.320463,4.944390e+09
min,-500.000000,3.200000e+01
25%,1.000000,1.101506e+09
50%,1.000000,9.200035e+09
75%,1.000000,9.200129e+09
max,2000.000000,9.204000e+10


In [4]:
df['StoreName'].value_counts().sort_values(ascending=False)

PM 4PM    358747
Name: StoreName, dtype: int64

In [5]:
df['Lv3Catagory'].value_counts().sort_values(ascending=False)

41000 - STATIONERY                 220584
45000 - VARIETIES                   50338
42000 - IT & GADGET ACCESSORIES     28800
44000 - HEALTH & BEAUTY             22131
43000 - HOME & DECORATIVES          12552
46000 - FASHION                      6435
53000 - SNACK / CONFECTIONERY        4988
68000 - OTHER INCOME                 4823
71000 - REDEMPTION                   3404
47000 - TRAVEL & SPORTS              1164
69000 - DISCOUNT                      961
54000 - ICE-CREAM                     733
52000 - BEVERAGE                      635
55000 - COFFEE & TEA                  279
65000 - GIFT  WRAP SERVICES           215
67000 - PAYMENT SERVICES               19
48000 - BOOK & MUSIC                   18
Name: Lv3Catagory, dtype: int64

In [6]:
def get_basket(df, index=['ReceiptId'], column='ItemName', limit=None):
    if limit is None:
        limit = df.shape[0]
    group = index.copy()
    group.append(column)
#     return (df.groupby(group)['Qty'].sum()
#             .reset_index()
#             .pivot_table(
#                 index=index,
#                 columns=columns,
#                 values=['Qty'],
#                 aggfunc='first')
#             .reset_index()
#             )
    return (df.groupby(group)['Qty'].sum()
            #     .reset_index().set_index(['ReceiptId', 'ItemName'])
            .iloc[:limit]
            .unstack()
            .reset_index().set_index(index)
            .fillna(0)
            )


In [7]:
# Convert the units to 1 hot encoded values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

def encode_basket(basket):
    tqdm.pandas(desc='encode units')
    # basket_sets = basket.progress_applymap(encode_units)
    basket_sets = basket.progress_applymap(encode_units)
    return basket_sets


In [8]:
# Filter StoreName
df = df[df['StoreName'] == 'PM 4PM']

## By Date

In [9]:
# Create basket
basket = get_basket(df, index=['Date'], column='Lv3Catagory')
basket_sets = encode_basket(basket)
display(basket_sets)

/Users/far/.pyenv/versions/miniconda3-4.3.30/envs/senior-proj-ai/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Lv3Catagory,41000 - STATIONERY,42000 - IT & GADGET ACCESSORIES,43000 - HOME & DECORATIVES,44000 - HEALTH & BEAUTY,45000 - VARIETIES,46000 - FASHION,47000 - TRAVEL & SPORTS,48000 - BOOK & MUSIC,52000 - BEVERAGE,53000 - SNACK / CONFECTIONERY,54000 - ICE-CREAM,55000 - COFFEE & TEA,65000 - GIFT WRAP SERVICES,67000 - PAYMENT SERVICES,68000 - OTHER INCOME,69000 - DISCOUNT,71000 - REDEMPTION
Date,,,,,,,,,,,,,,,,,
2016-03-24,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0
2016-03-25,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2016-03-26,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2016-03-27,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2016-03-28,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-10-24,1,1,1,1,1,1,0,0,1,1,0,0,0,0,1,0,0
2018-10-25,1,1,1,1,1,1,0,0,1,1,0,0,0,0,1,0,0
2018-10-26,1,1,1,1,1,1,1,0,1,1,0,0,0,0,1,0,0


In [10]:
# Build up the frequent items
frequent_itemsets = fpgrowth(basket_sets, min_support=0.05, use_colnames=True)
display(frequent_itemsets)

# Create the rules
# rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
display(rules)

,support,itemsets
0,1.000000,(41000 - STATIONERY)
1,0.997809,(42000 - IT & GADGET ACCESSORIES)
2,0.996714,(45000 - VARIETIES)
3,0.986857,(44000 - HEALTH & BEAUTY)
4,0.970427,(46000 - FASHION)
...,...,...
5050,0.065717,"(44000 - HEALTH & BEAUTY, 41000 - STATIONERY, ..."
5051,0.065717,"(45000 - VARIETIES, 44000 - HEALTH & BEAUTY, 4..."
5052,0.065717,"(45000 - VARIETIES, 44000 - HEALTH & BEAUTY, 4..."
5053,0.065717,"(45000 - VARIETIES, 44000 - HEALTH & BEAUTY, 4..."


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(41000 - STATIONERY),(42000 - IT & GADGET ACCESSORIES),1.000000,0.997809,0.997809,0.997809,1.000000,0.000000,1.000000
1,(42000 - IT & GADGET ACCESSORIES),(41000 - STATIONERY),0.997809,1.000000,0.997809,1.000000,1.000000,0.000000,inf
2,(41000 - STATIONERY),(45000 - VARIETIES),1.000000,0.996714,0.996714,0.996714,1.000000,0.000000,1.000000
3,(45000 - VARIETIES),(41000 - STATIONERY),0.996714,1.000000,0.996714,1.000000,1.000000,0.000000,inf
4,(42000 - IT & GADGET ACCESSORIES),(45000 - VARIETIES),0.997809,0.996714,0.995619,0.997805,1.001094,0.001088,1.496714
...,...,...,...,...,...,...,...,...,...
172388,"(52000 - BEVERAGE, 47000 - TRAVEL & SPORTS, 54...","(45000 - VARIETIES, 44000 - HEALTH & BEAUTY, 4...",0.088719,0.617744,0.065717,0.740741,1.199107,0.010912,1.474417
172389,"(47000 - TRAVEL & SPORTS, 54000 - ICE-CREAM, 6...","(45000 - VARIETIES, 44000 - HEALTH & BEAUTY, 4...",0.129244,0.363636,0.065717,0.508475,1.398305,0.018720,1.294671
172390,"(52000 - BEVERAGE, 54000 - ICE-CREAM, 68000 - ...","(45000 - VARIETIES, 44000 - HEALTH & BEAUTY, 4...",0.074480,0.794085,0.065717,0.882353,1.111156,0.006574,1.750274
172391,"(52000 - BEVERAGE, 54000 - ICE-CREAM, 68000 - ...","(45000 - VARIETIES, 44000 - HEALTH & BEAUTY, 4...",0.116101,0.441402,0.065717,0.566038,1.282363,0.014470,1.287204


## By Date and Member

In [11]:
# Create basket
basket = get_basket(df, index=['Date', 'AccountNum'], column='Lv3Catagory')
basket_sets = encode_basket(basket)
display(basket_sets)

/Users/far/.pyenv/versions/miniconda3-4.3.30/envs/senior-proj-ai/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Lv3Catagory              41000 - STATIONERY  42000 - IT & GADGET ACCESSORIES  \
Date       AccountNum                                                          
2016-03-24 3.476000e+03                   0                                1   
           6.070400e+04                   0                                0   
           1.128780e+05                   1                                0   
           1.284590e+05                   0                                0   
           1.414590e+05                   1                                0   
...                                     ...                              ...   
2018-10-28 9.203003e+09                   0                                0   
           9.203004e+09                   1                                0   
           9.203005e+09                   0                                0   
           9.203005e+09                   0                                1   
           9.203006e+09                   1                                0   

Lv3Catagory              43000 - HOME & DECORATIVES  44000 - HEALTH & BEAUTY  \
Date       AccountNum                                                          
2016-03-24 3.476000e+03                           0                        0   
           6.070400e+04                           0                        0   
           1.128780e+05                           0                        0   
           1.284590e+05                           0                        0   
           1.414590e+05                           0                        0   
...                                             ...                      ...   
2018-10-28 9.203003e+09                           0                        0   
           9.203004e+09                           0                        0   
           9.203005e+09                           0                        0   
           9.203005e+09                           0                        0   
           9.203006e+09                           0                        0   

Lv3Catagory              45000 - VARIETIES  46000 - FASHION  \
Date       AccountNum                                         
2016-03-24 3.476000e+03                  0                0   
           6.070400e+04                  1                0   
           1.128780e+05                  0                0   
           1.284590e+05                  1                0   
           1.414590e+05                  0                1   
...                                    ...              ...   
2018-10-28 9.203003e+09                  1                0   
           9.203004e+09                  0                0   
           9.203005e+09                  1                0   
           9.203005e+09                  0                0   
           9.203006e+09                  0                0   

Lv3Catagory              47000 - TRAVEL & SPORTS  48000 - BOOK & MUSIC  \
Date       AccountNum                                                    
2016-03-24 3.476000e+03                        0                     0   
           6.070400e+04                        0                     0   
           1.128780e+05                        0                     0   
           1.284590e+05                        0                     0   
           1.414590e+05                        0                     0   
...                                          ...                   ...   
2018-10-28 9.203003e+09                        0                     0   
           9.203004e+09                        0                     0   
           9.203005e+09                        0                     0   
           9.203005e+09                        0                     0   
           9.203006e+09                        0                     0   

Lv3Catagory              52000 - BEVERAGE  53000 - SNACK / CONFECTIONERY  \
Date       AccountNum                                      

In [12]:
# Build up the frequent items
frequent_itemsets = fpgrowth(basket_sets, min_support=0.005, use_colnames=True)
display(frequent_itemsets)

# Create the rules
# rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)
display(rules)

,support,itemsets
0,0.191193,(42000 - IT & GADGET ACCESSORIES)
1,0.258791,(45000 - VARIETIES)
2,0.594332,(41000 - STATIONERY)
3,0.045027,(46000 - FASHION)
4,0.108233,(44000 - HEALTH & BEAUTY)
5,0.072213,(43000 - HOME & DECORATIVES)
6,0.040240,(68000 - OTHER INCOME)
7,0.028349,(53000 - SNACK / CONFECTIONERY)
8,0.027868,(71000 - REDEMPTION)
9,0.007633,(47000 - TRAVEL & SPORTS)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(42000 - IT & GADGET ACCESSORIES, 45000 - VARI...",(41000 - STATIONERY),0.037318,0.594332,0.015496,0.415250,0.698683,-0.006683,0.693746
1,"(45000 - VARIETIES, 44000 - HEALTH & BEAUTY)",(41000 - STATIONERY),0.023504,0.594332,0.010440,0.444172,0.747346,-0.003529,0.729844
2,"(42000 - IT & GADGET ACCESSORIES, 44000 - HEAL...",(41000 - STATIONERY),0.016044,0.594332,0.007787,0.485321,0.816581,-0.001749,0.788195
3,"(43000 - HOME & DECORATIVES, 45000 - VARIETIES)",(41000 - STATIONERY),0.021129,0.594332,0.009767,0.462238,0.777744,-0.002791,0.754364
4,"(42000 - IT & GADGET ACCESSORIES, 43000 - HOME...",(41000 - STATIONERY),0.011853,0.594332,0.006162,0.519870,0.874713,-0.000883,0.844913
5,"(42000 - IT & GADGET ACCESSORIES, 43000 - HOME...",(45000 - VARIETIES),0.011853,0.258791,0.005229,0.441200,1.704851,0.002162,1.326430
6,"(43000 - HOME & DECORATIVES, 44000 - HEALTH & ...",(41000 - STATIONERY),0.016131,0.594332,0.007431,0.460667,0.775101,-0.002156,0.752166
7,(68000 - OTHER INCOME),(41000 - STATIONERY),0.040240,0.594332,0.018053,0.448638,0.754861,-0.005863,0.735756
8,(53000 - SNACK / CONFECTIONERY),(41000 - STATIONERY),0.028349,0.594332,0.014602,0.515090,0.866670,-0.002246,0.836583
9,(71000 - REDEMPTION),(41000 - STATIONERY),0.027868,0.594332,0.013698,0.491549,0.827061,-0.002864,0.797850
